# Run OpenQASM Benzene simulation

In [ ]:
from qdk import init, TargetProfile
from qdk.openqasm import compile, run
from qdk.widgets import Histogram

from pathlib import Path
src = Path("benzene_diradical_injected_rotation_measurement_circuit_0.qasm").read_text()

init(target_profile=TargetProfile.Base)
qir = compile(src)

## Show noiseless results from the existing sparse simulator

In [ ]:
# Run using the default sparse simulator
sim_results = run(src, shots=1024)
Histogram(sim_results, labels="kets")

## GPU simulation with noise
__Note:__ Simulating the high-level operations. _Not_ compiled to any specific target hardware

In [ ]:
from qdk.simulation import NoiseConfig
from qsharp._simulation import run_qir_gpu

# Use the GPU simulator with noise
noise = NoiseConfig()
noise.cx.set_depolarizing(0.0005)
noise.h.set_depolarizing(0.0005)
noise.s_adj.set_depolarizing(0.0005)
noise.rz.set_depolarizing(0.0007)
noise.mresetz.set_depolarizing(0.0002)

output = run_qir_gpu(qir, shots=1024, noise=noise, seed=None)
Histogram(output, labels="kets")

## View how this would run on an AC1000-like device

In [ ]:
from qdk.simulation import AC1000, NoiseConfig

ac1000 = AC1000()
ac1000.trace(qir)

## Simulate running on an AC1000-like device with noise

In [ ]:
noise = NoiseConfig()
noise.cz.set_depolarizing(0.05)
noise.sx.set_bitflip(0.01)
noise.mov.loss = 0.001
results = ac1000.simulate(qir, shots=1000, noise=noise, type="gpu")
Histogram(results, labels="kets", items="top-25")